In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import copy
import itertools
import networkx as nx
plt.rcParams['figure.figsize'] = (4, 4)

from qiskit.circuit.random import random_circuit
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit import *

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error, phase_amplitude_damping_error)
w = ''
from qiskit.visualization import plot_histogram, plot_gate_map

In [4]:
def q_num(qubit_coordinate, L):
    qubit_number = qubit_coordinate[1] + qubit_coordinate[0]*(2*L)
    return qubit_number

def q_list(q_num, L):
    q_list = np.array([int((q_num)//(2*L)),int(q_num%(2*L))])
    return q_list

def transform(coord, L):
    return coord[1] + coord[0]*(2*L)

In [5]:
##idenitify I = [0,0], Z = [0,1], X = [1,0], Y = [1,1]
def Had(err: np.ndarray) -> None:
    err[0], err[1] = err[1], err[0]


def cnot_X(control, target):
    control[1] ^= target[1]  # update control's Z: b → b ^ d
    target[0] ^= control[0]

#propagation of Z error, need to redo Ashutosh's code 
def cnot_Z(control, target):
    control[1] ^= target[0]  # update control's Z: b → b ^ c
    target[1] ^= control[0] 




def mes_Z(err):
    res_Z = 0
    if np.array_equal(err, np.array([1, 0])) or np.array_equal(err, np.array([1, 1])):
        res_Z += 1
    return res_Z

def mes_X(err):
    res_X = 0
    if np.array_equal(err, np.array([0, 1])) or np.array_equal(err, np.array([1, 1])):
        res_X += 1
    return res_X

    
def synZ(ancilla, errx1, errx2, errx3, errx4):
    ### errX1, err2, err3, err4 are data errors 
    ### err5 is the initialization error on the measurement qubit
    cnot_X(errx1, ancilla)
    cnot_X(errx2, ancilla)
    cnot_X(errx3, ancilla)
    cnot_X(errx4, ancilla)
   # res_Z  = mes_Z(ancilla)
    #return res_Z



def synX(ancilla, errz1, errz2, errz3, errz4):
    Had(ancilla)
    cnot_X(ancilla,errz1)
    cnot_X(ancilla,errz2)
    cnot_X(ancilla,errz3)
    cnot_X(ancilla, errz4)


def clock_Xi(my_dict, X_centers, X_stabs, i):
    for c, xc in enumerate(X_centers):
        cnot_X(my_dict[tuple(xc)], my_dict[tuple(X_stabs[c][i])])


def clock_Zi(my_dict, Z_centers, Z_stabs, i):
    for c, zc in enumerate(Z_centers):
        cnot_X(my_dict[tuple(Z_stabs[c][i])], my_dict[tuple(zc)])


In [6]:
# Now let us fully simulate the Toric Code using this approach 
# Start with just defining the Toric_Code class
class non_sym_Toric_Lattice:
    def __init__(self, L, k):
        self.dis = L
        self.asym = k
        self.size = 2*L
        self.array = [[None for _ in range(self.size)] for _ in range(self.size +self.asym)]
        self.initialize_coordinates_and_q_numbers()
    
    def get_nearest_neighbors(self, a, b):
        neighbors = [
            [(a - 1) % (self.size+ self.asym), b],  # Up 
            [a, (b - 1) % self.size],  # Left 
            [a, (b + 1) % self.size],   # Right
            [(a + 1) % (self.size+ self.asym), b],  # Down 
        ]
        return neighbors
    def initialize_coordinates_and_q_numbers(self):
        """
        Initialize coordinates and q_numbers for each position in the array.
        """
        for a in range(self.size + self.asym):
            for b in range(self.size):
                coordinate = [a, b]
                q_number = a * self.size + b
                if (a+b)%2 == 0:
                    self.array[a][b] = {
                        'coordinate': coordinate,
                        'q_number': q_number,
                        'category': 'phys'  }
                elif a%2==0:
                    self.array[a][b] = {
                        'coordinate': coordinate,
                        'q_number': q_number,
                        'category': 'X_ancilla',
                        'nn': self.get_nearest_neighbors(a,b)}                   ###### BIG NOTE: recently switched X and Z ancilla placement!
                        
                else:
                    self.array[a][b] = {
                        'coordinate': coordinate,
                        'q_number': q_number,
                        'category': 'Z_ancilla',
                        'nn': self.get_nearest_neighbors(a,b)}
    ####################################################################
####################################################################
    def get_dict(self):
        array_dict = {}
        for a in range(self.size + self.asym):
            for b in range(self.size):
                coordinate = (a, b)
                array_dict[coordinate] = self.array[a][b]
        return array_dict
        
    def get_sub_dicts(self, qbit_type):
        value_dict = self.get_dict()
        filtered_entries = [entry for entry in value_dict.values()
                    if entry['category'] in [qbit_type]]
        # Now we filter this filtered dictionary for the centers and edges of each category stabilizer
        return filtered_entries

    
    def get_matrix(self):
        matrix = np.zeros((self.size + self.asym, self.size), dtype=int)
        for a in range(self.size + self.asym):
            for b in range(self.size):
                category = self.array[a][b]['category']
                if category == 'Z_ancilla':
                    matrix[a][b] = 1
                elif category == 'X_ancilla':
                    matrix[a][b] = 2
        return matrix
    def zero_dict(self):
        # Now create a dictionary with err == [0,0] for all coordinates
        my_dict = {}
        for a in range(0, 2 * L + k):  
            for b in range(0, 2 * L):  
                key = [a, b]  
                my_dict[tuple(key)] = np.array([0, 0])  # Use tuple(key) because lists are not hashable  
        return my_dict


        return my_dict


In [7]:
# Then after having constructed the class/dictionary of the Code we will also rewrite the syndrome measurement operations
def measure_basic(Toric_Code, X_dict, Z_dict, my_dict):
    X_stabs   = [item['nn'] for item in X_dict]
    X_centers = [item['coordinate'] for item in X_dict]
    Z_stabs   = [item['nn'] for item in Z_dict]
    Z_centers = [item['coordinate'] for item in Z_dict]
    
    L = Toric_Code.dis
    k = Toric_Code.asym
    res_Z = np.zeros(len(Z_centers))
    res_X = np.zeros(len(X_centers))

    for i in range(0, 4):              # We do 4 cannonical time-steps to minimize error propagation
        clock_Xi(my_dict, X_centers, X_stabs, i)
        clock_Zi(my_dict, Z_centers, Z_stabs, i)
    
    # Then measure in appropriate basis and return results for analysis
    for i, center_num in enumerate(Z_centers):
        res_Z[i] = mes_Z(my_dict[tuple(center_num)])
    
    for i, center_num in enumerate(X_centers):
        res_X[i] = mes_X(my_dict[tuple(center_num)])

    return res_Z, res_X

In [8]:
def log_finder(L,k):
    log_h = np.zeros(2*L*(2*L +k), dtype=int)
    # Only assign ones on even indices in the range [0, 2*L].
    # Slicing from 0 to 2*L+1 with step 2 does that.
    log_h[:2*L :2] = 1
    log_v = np.zeros(2*L*(2*L +k), dtype=int)
    log_v[::2 * L +k] = 1
    
    return log_h, log_v



#### Need to taylor these to the qubit case here
def D3CSS_logX(my_dict, error_est, X_stabs, L_h, L_v):
    coords = []
    for coord, vec in my_dict.items():
        a, b = coord
        if tuple(vec) in ((1, 0), (1, 1)) and ((a + b) % 2 == 0):
            coords.append(coord)
    
    E_rc = list(set(coords) ^ set(error_est))
    E_r = np.zeros(2*L*(2*L +k), dtype = int)
    parity_S = False
    
    for row in E_rc:
        idx = transform(row, L)
        E_r[idx] = 1
    
    if (np.dot(L_h, E_r)%2 != 0) or (np.dot(L_v, E_r)%2 != 0):
        H_x = np.zeros(2*(L+k)*2*L)
        for row in X_stabs:
            for coord in row:
                idx = transform(coord, L)
                H_x[idx] = 1
        if np.dot(H_x, E_r)%2 != 0:
            parity_S = True             # This is the uncorrectable error btw
    return parity_S

def D3CSS_logZ(my_dict, error_est, Z_stabs, L_h, L_v):
    coords = []
    for coord, vec in my_dict.items():
        a, b = coord
        if tuple(vec) in ((0, 1), (1, 1)) and ((a + b) % 2 == 0):
            coords.append(coord)
    
    E_rc = list(set(coords) ^ set(error_est))
    E_r = np.zeros(2*L*(2*L +k), dtype = int)
    parity_S = False
    
    for row in E_rc:
        idx = transform(row, L)
        E_r[idx] = 1
    
    if (np.dot(L_h, E_r)%2 != 0) or (np.dot(L_v, E_r)%2 != 0):
        parity_S = True     # This is the uncorrectable error btw
    return parity_S
from random import randint


In [9]:
### New syndrome to defects function ----> this one keeps all results... 
def new_syndrome_to_defects(res_Z, res_X, Z_centers, X_centers):
    #### We want to through the array and count all places there are 1s and save them as defects
    
    synd_Z = {tuple(Z_centers[i]):value for i, value in enumerate(res_Z)}
    synd_X = {tuple(X_centers[i]):value for i, value in enumerate(res_X)}
    return synd_Z, synd_X 

In [10]:
### 3D DECODER FUNCTION ###
# takes as input the detection events
def decode_events(events):
        G = nx.Graph()
        for e in events:
            G.add_node(e)
        # fully connect events with weighted edges
        for i in range(len(events)):
            x1, y1, t1 = events[i]
            for j in range(i+1, len(events)):
                x2, y2, t2 = events[j]
                dx = min((x2 - x1) % (2*L), (x1 - x2) % (2*L))
                dy = min((y2 - y1) % (2*L), (y1 - y2) % (2*L))
                dt = abs(t2 - t1)
                weight = -(dx + dy + dt)    # negative weight for minimum weight matching
                G.add_edge(events[i], events[j], weight=weight)
        matches = nx.algorithms.matching.max_weight_matching(G, maxcardinality=True, weight='weight')
        return list(matches)

In [11]:
def correct_events(Z_matches, X_matches, L: int, rounds: int):
    # every entry will be a *list* of 3‑tuples, one list per edge
    corrections_X: List[Coord2] = []
    corrections_Z: List[Coord2] = []
    #### Wrap function decides whether we step up/down (t), left/right (y), forward/backward (x)
    def minimal_wrap(delta, size):
        delta %= size
        if delta >  size//2:
            delta -= size
        return delta
        
    ##### Safety function ####
    def _add(lst, x, y):
        if (x+y)%2==0:                
            lst.append((x % (2 * L), y % (2 * L)))
            
    # ---------- Z‑syndrome pairs   ->   X corrections ----------
    for e1, e2 in Z_matches:
        (x1, y1, t1), (x2, y2, t2) = e1, e2
        dx = minimal_wrap(x2 - x1, 2*L)
        dy = minimal_wrap(y2 - y1, 2*L)
        dt = t2 - t1        


        # --- time‑like moves (each uses a data qubit) ---
        step_t = 1 if dt >= 0 else -1
        for m in range(abs(dt)):
            qr = (x1) % (2*L)          # x stays the same
            qc = (y1) % (2*L)          # y stays the same
            qt = t1 + step_t*m
            _add(corrections_X, x1, y1)

        # --- x‑direction moves (horizontal qubits) ---
        i0 = (x1 - 1) // 2
        j0 = y1 // 2
        step_i = 1 if dx >= 0 else -1
        for m in range(abs(dx)//2):
            i = (i0 + step_i*m) % L
            qr = (2*i + 2) % (2*L) if step_i == 1 else (2*((i - 1) % L) + 2) % (2*L)
            qc = (2*j0) % (2*L)
            qt = t2                             # finish at the right time slice
            _add(corrections_X, qr, qc)

        # --- y‑direction moves (vertical qubits) ---
        step_j = 1 if dy >= 0 else -1
        for m in range(abs(dy)//2):
            j = (j0 + step_j*m) % L
            qc = (2*j + 1) % (2*L) if step_j == 1 else (2*((j - 1) % L) + 1) % (2*L)
            qr = (2*i0 + 1) % (2*L)
            qt = t2
            _add(corrections_X, qr, qc)

     
    for e1, e2 in X_matches:
        (x1, y1, t1), (x2, y2, t2) = e1, e2
        dx = minimal_wrap(x2 - x1, 2*L)
        dy = minimal_wrap(y2 - y1, 2*L)
        dt = minimal_wrap(t2 - t1, rounds)          

                                          

        # --- time‑like moves (each uses a data qubit) ---
        step_t = 1 if dt >= 0 else -1
        for m in range(abs(dt)):
            qr = (x1) % (2*L)          # x stays the same
            qc = (y1) % (2*L)          # y stays the same
            qt = (t1 + step_t*m) % rounds
            _add(corrections_Z, x1, y1)

        # --- x‑direction moves (horizontal qubits) ---
        i0 = (x1 - 1) // 2
        j0 = y1 // 2
        step_i = 1 if dx >= 0 else -1
        for m in range(abs(dx)//2):
            i = (i0 + step_i*m) % L
            qr = (2*i + 2) % (2*L) if step_i == 1 else (2*((i - 1) % L) + 2) % (2*L)
            qc = (2*j0) % (2*L)
            qt = t2                             # finish at the right time slice
            _add(corrections_Z, qr, qc)

        # --- y‑direction moves (vertical qubits) ---
        step_j = 1 if dy >= 0 else -1
        for m in range(abs(dy)//2):
            j = (j0 + step_j*m) % L
            qc = (2*j + 1) % (2*L) if step_j == 1 else (2*((j - 1) % L) + 1) % (2*L)
            qr = (2*i0 + 1) % (2*L)
            qt = t2
            _add(corrections_Z, qr, qc)



    return corrections_X, corrections_Z


In [12]:
### ERROR APPLICATION FUNCTION ###
# py functionality still wanting, maybe(?)
def XYZ_noise(zero_dict, px, py, pz, stab_noise):
    Error_X_list = []
    Error_Y_list = []
    Error_Z_list = []
    
    
    # APPLY NOISE on qbits
    for q in qbits:
        random_numx  = np.random.choice([0,1], p = [1-px, px])
        random_numz = np.random.choice([0,1], p = [1 -pz, pz])
        random_numy = np.random.choice([0,1], p = [1 - py, py])
        if random_numx == 1:
            Error_X_list.append(tuple(q['coordinate']))    # for now just apply bit-flip errors
        if random_numz == 1:
            Error_Z_list.append(tuple(q['coordinate']))
        if random_numy == 1:
            Error_Y_list.append(tuple(q['coordinate']))

    # APPLY NOISE on ancillas
    if stab_noise:
        for xstab in X_stabs:
            for xc in xstab:
                random_numz = np.random.choice([0,1], p=[1-pz, pz])
                random_numy = np.random.choice([0,1], p = [1-py, py])
                if random_numz == 1:
                    Error_Z_list.append(tuple(xc))
                if random_numy ==1:
                    Error_Y_list.append(xc)
    
        for zstab in Z_stabs: 
            for zc in zstab:
                random_numx = np.random.choice([0,1], p=[1-px, px])
                random_numy = np.random.choice([0,1], p = [1-py, py])
                if random_numx == 1:
                    Error_X_list.append(tuple(zc))
                if random_numy == 1:
                    Error_Y_list.append(tuple(zc))
        
    for err in Error_X_list:
        zero_dict[err] = (zero_dict[tuple(err)] + np.array([1,0]))%2
    for err in Error_Z_list:
        zero_dict[err] = (zero_dict[tuple(err)] + np.array([0,1]))%2
    for err in Error_Y_list:
        zero_dict[err] = (zero_dict[tuple(err)] + np.array([1,1]))%2
    
    return Error_X_list, Error_Y_list, Error_Z_list

Run the following code cell to turn on uniform noise on the control not operations

In [ ]:
# Introduce new errored functions instead of making all functions errored at the outset
def cnot_X_pcnot(control, target, p_cnot):
    control[1] ^= target[1]
    target[0] ^= control[0]

    if random.random() < p_cnot:
        n = random.randint(1,3)
        list_err = np.array([int(bit) for bit in f"{n:02b}"])
        #n = random.randint(1, 2**4 - 1)
        #list_err = np.array([int(bit) for bit in f"{n:04b}"])
        control[0] ^= list_err[0]
        #control[1] ^= list_err[1]
        target[0] ^= list_err[1]
        #target[1] ^= list_err[1]
    
    return control, target

def clock_Xi_pcnot(my_dict, X_centers, X_stabs, i, p_cnot):
    for c, xc in enumerate(X_centers):
        cnot_X_pcnot(my_dict[tuple(xc)], my_dict[tuple(X_stabs[c][i])], p_cnot)
        

def clock_Zi_pcnot(my_dict, Z_centers, Z_stabs, i, p_cnot):
    for c, zc in enumerate(Z_centers):
        cnot_X_pcnot(my_dict[tuple(Z_stabs[c][i])], my_dict[tuple(zc)], p_cnot)

def measure_basic_pcnot(Toric_Code, X_dict, Z_dict, my_dict, p_cnot):
    X_stabs   = [item['nn'] for item in X_dict]
    X_centers = [item['coordinate'] for item in X_dict]
    Z_stabs   = [item['nn'] for item in Z_dict]
    Z_centers = [item['coordinate'] for item in Z_dict]
    
    L = Toric_Code.dis
    k = Toric_Code.asym
    res_Z = np.zeros(len(Z_centers))
    res_X = np.zeros(len(X_centers))

    for i in range(0, 4):              # We do 4 cannonical time-steps to minimize error propagation
        clock_Xi_pcnot(my_dict, X_centers, X_stabs, i, p_cnot)
        clock_Zi_pcnot(my_dict, Z_centers, Z_stabs, i, p_cnot)
    
    # Then measure in appropriate basis and return results for analysis
    for i, center_num in enumerate(Z_centers):
        res_Z[i] = mes_Z(my_dict[tuple(center_num)])
    
    for i, center_num in enumerate(X_centers):
        res_X[i] = mes_X(my_dict[tuple(center_num)])

    return res_Z, res_X